# Homework

Привет! В этой домашнем задании ты научишься обучении модели BERT. На семинаре был разобран код модели, здесь же посмотрим на то, как надо обработать данные, чтобы на них модель могла учиться. 

Замечания по выполнению задания:

- Код внутри блока `<DON'T TOUCH THIS!>` используется для проверки задания, его нельзя трогать. 

- Внутри блока `<YOUR CODE>` может больше кода, чем там показано изначально.

- От задания требуется написания небольшого отчета в конце.


Для начала загрузи нужные библиотеки.

In [1]:
!pip install transformers catalyst

    100% |████████████████████████████████| 645kB 1.8MB/s ta 0:00:01
    100% |████████████████████████████████| 348kB 1.8MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/e1/d2/6f02df2616fd4016075f60157c7a0452b38d8f7938ae94343911e0fb0b09/dataclasses-0.7-py3-none-any.whl
    100% |████████████████████████████████| 3.8MB 443kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/03/27/e35e7c6e6a52fab9fcc64fc2b20c6b516eba930bb02b10ace3b38200d3ab/numpy-1.18.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/1a/70/1935c770cb3be6e3a8b78ced23d7e0f3b187f5cbfab4749523ed65d7c9b1/requests-2.23.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/83/71a2ee6158bb9f39a90c0dea1637f81d5eef866e188e1971a1b1ab01a35a/filelock-3.0.12-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c9/40/058b12e8ba10e35f89c9b1fdfc2d4c7f8c05947df2d5eb3c7b258019fda0/tqdm-4.46.0-py2.py3-none-any.whl


  Using cached https://files.pythonhosted.org/packages/a0/df/635cdb901ee4a8a42ec68e480c49f85f4c59e8816effbf57d9e6ee8b3588/setuptools-46.1.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ed/1b/72a1821152d07cf1d8b6fce298aeb06a7eb90f4d6d41acec9861e7cc6df0/decorator-4.4.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2d/68/106af3ae51daf807e9cdcba6a90e518954eb8b70341cee52995540a53ead/Pygments-2.6.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352eef9db37c49445d1fec35509022f6aa/scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/48/11/d1800bca0a3bae820b84b7d813ad1eff15a48a64caea9c823fc8c1b119e8/gitdb-4.0.5-py3-none-any.whl
    100% |████████████████████████████████| 92kB 4.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/8c/23/848298cccf8e40f5bbb59009b32848a4c38f4e7f3364297ab3c3e2e2cd14/wheel-0.34.2-py2.py3-

In [1]:
import os
import random
import sys
import urllib.request
import zipfile

import numpy as np
import pandas as pd
from tqdm import tqdm, trange

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, Dataset

import transformers

from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import AccuracyCallback, SchedulerCallback, F1ScoreCallback
from catalyst.utils import set_global_seed, prepare_cudnn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



Внизу идет технический код, который нужен для загрузки датасетов. Его можно уменьшить, выбрав только некоторые из них. Для того, что бы зачесть задание, надо выбрать не менее двух задач, для хотя бы одной из которых нужно использовать два предложения(ответ и вопрос, два предложения и прочее). Подробнее про датасеты [здесь](https://gluebenchmark.com/).

In [2]:
# TASKS = ["CoLA", "SST", "MRPC", "QQP", "STS", "MNLI", "SNLI", "QNLI", "RTE", "WNLI"]
# TASK2PATH = {
#     "CoLA": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FCoLA.zip?alt=media&token=46d5e637-3411-4188-bc44-5809b5bfb5f4",
#     "SST": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8",
#     "MRPC": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc",
#     "QQP": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQQP.zip?alt=media&token=700c6acf-160d-4d89-81d1-de4191d02cb5",
#     "STS": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSTS-B.zip?alt=media&token=bddb94a7-8706-4e0d-a694-1109e12273b5",
#     "MNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FMNLI.zip?alt=media&token=50329ea1-e339-40e2-809c-10c40afff3ce",
#     "SNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSNLI.zip?alt=media&token=4afcfbb2-ff0c-4b2d-a09a-dbf07926f4df",
#     "QNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQNLIv2.zip?alt=media&token=6fdcf570-0fc5-4631-8456-9505272d1601",
#     "RTE": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FRTE.zip?alt=media&token=5efa7e85-a0bb-4f19-8ea2-9e1840f077fb",
#     "WNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FWNLI.zip?alt=media&token=068ad0a0-ded7-4bd7-99a5-5e00222e0faf",
# }

# MRPC_TRAIN = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt"
# MRPC_TEST = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt"

# data_dir = "data/"
# max_seq_length = 128

In [3]:
TASKS = ["SST", "QQP"]
TASK2PATH = {
    "SST": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8",
    "QQP": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQQP.zip?alt=media&token=700c6acf-160d-4d89-81d1-de4191d02cb5"
}

MRPC_TRAIN = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt"
MRPC_TEST = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt"

data_dir = "data/"
max_seq_length = 128

In [4]:
def download_and_extract(task, data_dir):
    print("Downloading and extracting %s..." % task)
    data_file = "%s.zip" % task
    urllib.request.urlretrieve(TASK2PATH[task], data_file)
    with zipfile.ZipFile(data_file) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(data_file)
    print("\tCompleted!")

def format_mrpc(data_dir, path_to_data):
    print("Processing MRPC...")
    mrpc_dir = os.path.join(data_dir, "MRPC")
    if not os.path.isdir(mrpc_dir):
        os.mkdir(mrpc_dir)
    if path_to_data:
        mrpc_train_file = os.path.join(path_to_data, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(path_to_data, "msr_paraphrase_test.txt")
    else:
        print("Local MRPC data not specified, downloading data from %s" % MRPC_TRAIN)
        mrpc_train_file = os.path.join(mrpc_dir, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(mrpc_dir, "msr_paraphrase_test.txt")
        urllib.request.urlretrieve(MRPC_TRAIN, mrpc_train_file)
        urllib.request.urlretrieve(MRPC_TEST, mrpc_test_file)
    assert os.path.isfile(mrpc_train_file), "Train data not found at %s" % mrpc_train_file
    assert os.path.isfile(mrpc_test_file), "Test data not found at %s" % mrpc_test_file
    urllib.request.urlretrieve(TASK2PATH["MRPC"], os.path.join(mrpc_dir, "dev_ids.tsv"))

    dev_ids = []
    with open(os.path.join(mrpc_dir, "dev_ids.tsv"), encoding="utf8") as ids_fh:
        for row in ids_fh:
            dev_ids.append(row.strip().split("\t"))

    with open(mrpc_train_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "train.tsv"), "w", encoding="utf8"
    ) as train_fh, open(os.path.join(mrpc_dir, "dev.tsv"), "w", encoding="utf8") as dev_fh:
        header = data_fh.readline()
        train_fh.write(header)
        dev_fh.write(header)
        for row in data_fh:
            label, id1, id2, s1, s2 = row.strip().split("\t")
            if [id1, id2] in dev_ids:
                dev_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))
            else:
                train_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))

    with open(mrpc_test_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "test.tsv"), "w", encoding="utf8"
    ) as test_fh:
        header = data_fh.readline()
        test_fh.write("index\t#1 ID\t#2 ID\t#1 String\t#2 String\n")
        for idx, row in enumerate(data_fh):
            label, id1, id2, s1, s2 = row.strip().split("\t")
            test_fh.write("%d\t%s\t%s\t%s\t%s\n" % (idx, id1, id2, s1, s2))
    print("\tCompleted!")

In [5]:
TASKS = ["SST", "QQP"] # Или можно просто сюда вписать те датасеты, которые ты выбрал.

for task in TASKS:
    if task == "MRPC":
        format_mrpc(data_dir, None)
    else:
        download_and_extract(task, data_dir)

	Completed!
	Completed!


Загрузи один из выбранных датасет с помощью Pandas(не обязательно через него, но так проще) и посмотри на него.

In [92]:
data = pd.read_csv("./data/SST-2/train.tsv", sep="\t")
data.describe()
# 67349

,label
count,67349.000000
mean,0.557826
std,0.496649
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [93]:
train_pd, valid_pd = train_test_split(data, test_size=0.2)
test_pd = pd.read_csv("./data/SST-2/dev.tsv", sep="\t")
test_pd.head()

,sentence,label
0,it 's a charming and often affecting journey .,1
1,unflinchingly bleak and desperate,0
2,allows us to hope that nolan is poised to emba...,1
3,"the acting , costumes , music , cinematography...",1
4,"it 's slow -- very , very slow .",0


Для начала рассмотрим важную часть обработки текста для трансфомера(и не только) – токенайзер.

В качестве примера токенайзера воспользуемся внутренним из библиотеки transformers, обученным для BERT-а. Посмотрим, что он умеет.

In [9]:
model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Посмотрим, как происходит токенизация предложения.

In [10]:
test_sentence = "Hide new secretions from the parental units."
print(tokenizer.tokenize(test_sentence))

['hide', 'new', 'secret', '##ions', 'from', 'the', 'parental', 'units', '.']


Видно, что предложения разделяются не на слова, а подслова. Токены, которые надо объеденить в слова для получения "нормального" текста, выделены с помощью `##`. Посмотрим, как различаются коды токенов с этим символом и без него.

In [11]:
print(tokenizer.convert_tokens_to_ids(['ions']))
print(tokenizer.convert_tokens_to_ids(['##ions']))

[15956]
[8496]


Для токенизации предложений воспользуемся методом `encode`. Она принимает предложение как строку или список токенов**(!)**.

In [12]:
print(tokenizer.encode(test_sentence))

[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 1012, 102]


Добавились специальные токены впереди и сзади предложения. Посмотрим на весь список специальных токенов:

In [13]:
print(tokenizer.special_tokens_map)
print({i: j for i, j in zip(tokenizer.all_special_tokens, tokenizer.all_special_ids)})

{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
{'[UNK]': 100, '[CLS]': 101, '[MASK]': 103, '[SEP]': 102, '[PAD]': 0}


Посмотрим, что ещё может делать токенайзер. Что требуется нам для обучения BERT-а: добавить паддинг, получить маску аттеншена и тип токенов. Попробуем сделать это самостоятельно и посмотрим, как это сделать с помощью токенайзера.

Выбери два предложения из обучающей выборки. Получи их токены с помощью метода `tokenize`. Объедени списки токенов так, чтобы модель могла различать, что они от разных предложений. 

(Подсказка: на семинаре была картинка с эмбеддингами. Она может подсказать, что надо изменить в токенах предложения) 

In [14]:
train_pd.iloc[0:2].sentence[0]
# train_pd.iloc[1].sentence

'hide new secretions from the parental units '

In [15]:
# <YOUR CODE>
s1, s2 = train_pd.iloc[0].sentence, train_pd.iloc[1].sentence
tokenized_s1, tokenized_s2 = tokenizer.encode(s1), tokenizer.encode(s2)
tokenized_union = tokenized_s1 + tokenized_s2[1:] 
# </YOUR CODE>

# <DON'T TOUCH THIS!>
assert tokenizer.encode(s_union) == tokenizer.encode(s1, s2), "Not equal"
# </DON'T TOUCH THIS!>

NameError: name 's_union' is not defined

Теперь надо добавь нулей в полученный список чисел, чтобы они легко складывались в батчи.

In [ ]:
max_seq_length = 128
encoded_full = tokenized_union + [0] * (max_seq_length - len(tokenized_union))

In [ ]:
max_seq_length = 128
# <YOUR CODE>
encoded_full = tokenized_union + [0] * (max_seq_length - len(tokenized_union))
# </YOUR CODE>

# <DON'T TOUCH THIS!>
encoded_correct = tokenizer.encode(s1, s2, max_length=max_seq_length, pad_to_max_length=True)
assert len(encoded_full) == len(encoded_correct), "Different length"
assert encoded_full == encoded_correct, "Not equal"
# </DON'T TOUCH THIS!>

В модель также надо кинуть маску для механизма внимания и тип предложения для каждого токена. Сделай их.

In [ ]:
# <YOUR CODE>
token_type_ids = [int(i > encoded_full.index(102) and x != 0) for i, x in enumerate(encoded_full) ]
attention_mask = [ int(i!=0) for i in encoded_full ]
# </YOUR CODE>

# <DON'T TOUCH THIS!>
encoded_plus = tokenizer.encode_plus(train['sentence'][0], text_pair=train['sentence'][1], max_length=max_seq_length, pad_to_max_length=True)
assert len(token_type_ids) == len(encoded_plus['token_type_ids']), "Different length in token_type_ids"
assert token_type_ids == encoded_plus['token_type_ids'], "Not equal token_type_ids"
assert len(attention_mask) == len(encoded_plus['attention_mask']), "Different length in attention_mask"
assert attention_mask == encoded_plus['attention_mask'], "Not equal attention_mask"
# </DON'T TOUCH THIS!>

Как видно из тестов, все нужные для обработки текста для BERT-а вещи может делать токенизатор из `transformers`. Но не все токенизаторы настолько функциональны. Их (почти)полный список:
- [Sentence Piece](https://github.com/google/sentencepiece/)
- [fastBPE](https://github.com/glample/fastBPE)
- [Hugging Face Tokenizers](https://github.com/huggingface/tokenizers)
- [YouTokenToMe](https://github.com/VKCOM/YouTokenToMe)

Их сравнивают [здесь](https://github.com/VKCOM/YouTokenToMe/blob/master/benchmark.md) или [здесь](https://towardsdatascience.com/a-small-timing-experiment-on-the-new-tokenizers-library-a-write-up-7caab6f80ea6). Также специальные токенайзеры, которые специализируются на "незападные" языки. Но не будем на них останавливаться.

Теперь ты знаешь достаточно, чтобы написать обработчик данных. Что надо сделать: получить из данных предложения, закодировать их, получить аттенш маску и тип токенов, не забыть про таргет. 

P.S. Есть более быстрая версия токенизатора для BERT внутри `transformers`, `BertTokenizerFast`. 

P.S.S. Теперь надо использовать только функционал токенайзера для кодирования предложений, без велосипедов.

In [94]:
class TextClassificationDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

        # <YOUR CODE>
        self.encoded_plus = [ 
            tokenizer.encode_plus(item, max_length=max_seq_length, pad_to_max_length=True) 
            for item in data['sentence']
        ] 
        
#         print(self.encoded_plus[0])
        
        self.input_ids = torch.tensor([ i['input_ids'] for i in self.encoded_plus ], dtype=torch.long) 
        self.attention_mask = torch.tensor([ i['attention_mask'] for i in self.encoded_plus ], dtype=torch.long) 
        self.token_type_ids = torch.tensor([ i['token_type_ids'] for i in self.encoded_plus ], dtype=torch.long) 
        self.target = torch.tensor(list(self.data['label']), dtype=torch.long) 
        # </YOUR CODE>

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'token_type_ids': self.token_type_ids[idx],
            'targets': self.target[idx]
        }

Воспользуйтесь семинаром и построй модель для классификации предложений.

(Подсказка: весь код BERT-а из семинара доступен из библиотеки `transformers`)

In [95]:
class BertForSequenceClassification(nn.Module):
    def __init__(self, pretrained_model_name: str, num_labels: int):
        super().__init__()

        # <YOUR CODE>
        
        config = transformers.AutoConfig.from_pretrained(
            pretrained_model_name,
            num_labels = num_labels
        )
        
        self.bert = transformers.BertModel.from_pretrained(
            pretrained_model_name, 
            num_labels=num_labels
        )
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # </YOUR CODE>

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        
        assert attention_mask is not None, "attention mask is none"
        bert_output = self.bert(input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids)
        hidden_state = bert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        
        # <YOUR CODE>
        logits = self.classifier(pooled_output)  # (bs, dim)
        # </YOUR CODE>

        return logits

Выбери из [списка](https://huggingface.co/models?search=google%2Fbert_) несколько моделей, которые ты будешь обучать. Сравни их качество на выбранных датасетах. 

Лучше всего будет выбрать одну основную конфигурацию, и другие с небольшим изменением. Например, пройтись по такой сетке: `{'layers': [2, 4], 'num_heads': [2, 4]}`. 

# Используем google/bert_uncased_L-4_H-256_A-4 на SST-2

In [96]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 2

# <YOUR CODE>
pretrained_model_name = 'google/bert_uncased_L-4_H-256_A-4'
tokenizer = transformers.BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification(pretrained_model_name, num_labels=num_labels)
# </YOUR CODE>

model.to(device)
print("Success!")

Success!


In [97]:
batch_size = 32


# <YOUR CODE>
train_dataset = TextClassificationDataset(train_pd, tokenizer)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

valid_dataset = TextClassificationDataset(valid_pd, tokenizer)
valid_sampler = RandomSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=batch_size)

test_dataset = TextClassificationDataset(test_pd, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

dataloaders = {
    "train": train_dataloader,
    "valid": valid_dataloader,
    "test": test_dataloader    
}
# </YOUR CODE>

print(f"Dataset size: {len(train_dataloader)}")

Dataset size: 1684


In [98]:
seed = 404
set_global_seed(seed)
prepare_cudnn(True)

In [99]:
# Гиперпараметры для обучения модели. Подбери нужные для каждой модели.

epochs = 10
lr = 1e-5
warmup_steps = len(train_dataloader) // 2
t_total = len(train_dataloader) * epochs

Добавь Loss, Optimizer и Scheduler.

In [100]:
optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters()], "weight_decay": 0.0},
]

# <YOUR CODE>
criterion = torch.nn.CrossEntropyLoss()
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)
# </YOUR CODE>

In [101]:
log_dir = 'logs/'

Для обучения модели воспользуемся библиотекой `catalyst`.

In [102]:
train_val_loaders = {
    "train": train_dataloader,
    "valid": valid_dataloader
}

runner = SupervisedRunner(
    input_key=(
        "input_ids",
        "attention_mask",
        "token_type_ids"
    )
)

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=train_val_loaders,
    callbacks=[
        AccuracyCallback(num_classes=num_labels),
        SchedulerCallback(mode='batch'),
    ],
    logdir=log_dir,
    num_epochs=epochs,
    verbose=False,
)

/home/care1e55/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



[2020-05-11 15:17:33,862] 
1/10 * Epoch 1 (_base): lr=9.474e-06 | momentum=0.9000
1/10 * Epoch 1 (train): accuracy01=0.7167 | loss=0.5280 | lr=7.371e-06 | momentum=0.9000
1/10 * Epoch 1 (valid): accuracy01=0.8319 | loss=0.3831


/home/care1e55/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning:

Please also save or load the state of the optimzer when saving or loading the scheduler.



[2020-05-11 15:20:38,030] 
2/10 * Epoch 2 (_base): lr=8.421e-06 | momentum=0.9000
2/10 * Epoch 2 (train): accuracy01=0.8489 | loss=0.3470 | lr=8.947e-06 | momentum=0.9000
2/10 * Epoch 2 (valid): accuracy01=0.8704 | loss=0.3078
[2020-05-11 15:23:46,565] 
3/10 * Epoch 3 (_base): lr=7.368e-06 | momentum=0.9000
3/10 * Epoch 3 (train): accuracy01=0.8744 | loss=0.2965 | lr=7.894e-06 | momentum=0.9000
3/10 * Epoch 3 (valid): accuracy01=0.8841 | loss=0.2806
[2020-05-11 15:26:50,754] 
4/10 * Epoch 4 (_base): lr=6.316e-06 | momentum=0.9000
4/10 * Epoch 4 (train): accuracy01=0.8925 | loss=0.2598 | lr=6.842e-06 | momentum=0.9000
4/10 * Epoch 4 (valid): accuracy01=0.8944 | loss=0.2587
[2020-05-11 15:29:54,739] 
5/10 * Epoch 5 (_base): lr=5.263e-06 | momentum=0.9000
5/10 * Epoch 5 (train): accuracy01=0.9040 | loss=0.2360 | lr=5.789e-06 | momentum=0.9000
5/10 * Epoch 5 (valid): accuracy01=0.9039 | loss=0.2470
[2020-05-11 15:32:58,738] 
6/10 * Epoch 6 (_base): lr=4.211e-06 | momentum=0.9000
6/10 * Epo

In [103]:
logits = runner.predict_loader(model=model, loader=test_dataloader, resume=log_dir+'checkpoints/best.pth')
y_pred = []
counter = 0
softmax = torch.nn.Softmax(dim=1)
for i in logits:
    for j in softmax(i['logits']).argmax(axis=1).tolist():
        y_pred.append(j)

y_true = []

for i in test_dataloader:
    for j in i['targets'].tolist():
        y_true.append(j)
        

accuracy_score(y_true, y_pred)

0.8555045871559633

# Используем google/bert_uncased_L-2_H-128_A-2 на SST-2

In [105]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 2

# <YOUR CODE>
pretrained_model_name = 'google/bert_uncased_L-2_H-128_A-2'
tokenizer = transformers.BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification(pretrained_model_name, num_labels=num_labels)
# </YOUR CODE>

model.to(device)
print("Success!")

# Гиперпараметры для обучения модели. Подбери нужные для каждой модели.

epochs = 10
lr = 1e-5
warmup_steps = len(train_dataloader) // 2
t_total = len(train_dataloader) * epochs

optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters()], "weight_decay": 0.0},
]

# <YOUR CODE>
criterion = torch.nn.CrossEntropyLoss()
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)
# </YOUR CODE>

Success!


In [106]:
train_val_loaders = {
    "train": train_dataloader,
    "valid": valid_dataloader
}

runner = SupervisedRunner(
    input_key=(
        "input_ids",
        "attention_mask",
        "token_type_ids"
    )
)

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=train_val_loaders,
    callbacks=[
        AccuracyCallback(num_classes=num_labels),
        SchedulerCallback(mode='batch'),
    ],
    logdir=log_dir,
    num_epochs=epochs,
    verbose=False,
)

/home/care1e55/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



[2020-05-11 15:48:27,478] 
1/10 * Epoch 1 (_base): lr=9.474e-06 | momentum=0.9000
1/10 * Epoch 1 (train): accuracy01=0.6029 | loss=0.6739 | lr=7.371e-06 | momentum=0.9000
1/10 * Epoch 1 (valid): accuracy01=0.7538 | loss=0.5087


/home/care1e55/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning:

Please also save or load the state of the optimzer when saving or loading the scheduler.



[2020-05-11 15:49:17,749] 
2/10 * Epoch 2 (_base): lr=8.421e-06 | momentum=0.9000
2/10 * Epoch 2 (train): accuracy01=0.7770 | loss=0.4663 | lr=8.947e-06 | momentum=0.9000
2/10 * Epoch 2 (valid): accuracy01=0.8232 | loss=0.3970
[2020-05-11 15:50:07,414] 
3/10 * Epoch 3 (_base): lr=7.368e-06 | momentum=0.9000
3/10 * Epoch 3 (train): accuracy01=0.8188 | loss=0.4004 | lr=7.894e-06 | momentum=0.9000
3/10 * Epoch 3 (valid): accuracy01=0.8428 | loss=0.3633
[2020-05-11 15:50:57,017] 
4/10 * Epoch 4 (_base): lr=6.316e-06 | momentum=0.9000
4/10 * Epoch 4 (train): accuracy01=0.8374 | loss=0.3670 | lr=6.842e-06 | momentum=0.9000
4/10 * Epoch 4 (valid): accuracy01=0.8547 | loss=0.3405
[2020-05-11 15:51:46,601] 
5/10 * Epoch 5 (_base): lr=5.263e-06 | momentum=0.9000
5/10 * Epoch 5 (train): accuracy01=0.8496 | loss=0.3434 | lr=5.789e-06 | momentum=0.9000
5/10 * Epoch 5 (valid): accuracy01=0.8621 | loss=0.3277
[2020-05-11 15:52:36,146] 
6/10 * Epoch 6 (_base): lr=4.211e-06 | momentum=0.9000
6/10 * Epo

In [107]:
logits = runner.predict_loader(model=model, loader=test_dataloader, resume=log_dir+'checkpoints/best.pth')
y_pred = []
counter = 0
softmax = torch.nn.Softmax(dim=1)
for i in logits:
    for j in softmax(i['logits']).argmax(axis=1).tolist():
        y_pred.append(j)

y_true = []

for i in test_dataloader:
    for j in i['targets'].tolist():
        y_true.append(j)
        

accuracy_score(y_true, y_pred)

0.805045871559633

# Используем google/bert_uncased_L-2_H-128_A-2 на QQP

In [82]:
data = pd.read_csv("./data/QQP/train.tsv", sep="	", error_bad_lines=False).dropna()
data.describe()
# 	363185

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/home/care1e55/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.



,qid2,is_duplicate
count,363177.000000,363177.000000
mean,220914.009747,0.369354
std,159920.121426,0.482631
min,2.000000,0.000000
25%,74634.000000,0.000000
50%,196944.000000,0.000000
75%,354683.000000,1.000000
max,537933.000000,1.000000


In [83]:
from sklearn.model_selection import train_test_split
train_pd, valid_pd = train_test_split(data, test_size=0.2)
test_pd = pd.read_csv("./data/QQP/dev.tsv", sep="\t").dropna()
test_pd.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,201359,303345,303346,Why are African-Americans so beautiful?,Why are hispanics so beautiful?,0.0
1,263843,69383,380476,I want to pursue PhD in Computer Science about...,I handle social media for a non-profit. Should...,0.0
2,172974,266948,175089,Is there a reason why we should travel alone?,What are some reasons to travel alone?,1.0
3,15329,29298,29299,Why are people so obsessed with having a girlf...,How can a single male have a child?,0.0
4,209794,314169,314170,What are some good baby girl names starting wi...,What are some good baby girl names starting wi...,0.0


In [84]:
class TextClassificationDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

        # <YOUR CODE>
        self.encoded_plus = [ 
            tokenizer.encode_plus(text=item[0],text_pair=item[1] , max_length=max_seq_length, pad_to_max_length=True) 
            for item in zip(self.data['question1'], self.data['question2'])
        ] 
        
#         print(self.encoded_plus[0])
        
        self.input_ids = torch.tensor([ i['input_ids'] for i in self.encoded_plus ], dtype=torch.long) 
        self.attention_mask = torch.tensor([ i['attention_mask'] for i in self.encoded_plus ], dtype=torch.long) 
        self.token_type_ids = torch.tensor([ i['token_type_ids'] for i in self.encoded_plus ], dtype=torch.long) 
        self.target = torch.tensor(list(self.data['is_duplicate']), dtype=torch.long) 
        # </YOUR CODE>

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'token_type_ids': self.token_type_ids[idx],
            'targets': self.target[idx]
        }

In [85]:
class BertForSequenceClassification(nn.Module):
    def __init__(self, pretrained_model_name: str, num_labels: int):
        super().__init__()

        # <YOUR CODE>
        
        config = transformers.AutoConfig.from_pretrained(
            pretrained_model_name,
            num_labels = num_labels
        )
        
        self.bert = transformers.BertModel.from_pretrained(
            pretrained_model_name, 
            num_labels=num_labels
        )
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # </YOUR CODE>

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        
        assert attention_mask is not None, "attention mask is none"
        bert_output = self.bert(input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids)
        hidden_state = bert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        
        # <YOUR CODE>
        logits = self.classifier(pooled_output)  # (bs, dim)
        # </YOUR CODE>

        return logits

In [86]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 2

# <YOUR CODE>
pretrained_model_name = 'google/bert_uncased_L-2_H-128_A-2'
tokenizer = transformers.BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification(pretrained_model_name, num_labels=num_labels)
# </YOUR CODE>

model.to(device)
print("Success!")

Success!


In [87]:
batch_size = 32

# <YOUR CODE>
train_dataset = TextClassificationDataset(train_pd, tokenizer)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

valid_dataset = TextClassificationDataset(valid_pd, tokenizer)
valid_sampler = RandomSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=batch_size)

test_dataset = TextClassificationDataset(test_pd, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

dataloaders = {
    "train": train_dataloader,
    "valid": valid_dataloader,
    "test": test_dataloader    
}
# </YOUR CODE>

print(f"Dataset size: {len(train_dataloader)}")

Dataset size: 9080


In [88]:
seed = 404
set_global_seed(seed)
prepare_cudnn(True)
# Гиперпараметры для обучения модели. Подбери нужные для каждой модели.

epochs = 10
lr = 1e-5
warmup_steps = len(train_dataloader) // 2
t_total = len(train_dataloader) * epochs

optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters()], "weight_decay": 0.0},
]

# <YOUR CODE>
criterion = torch.nn.CrossEntropyLoss()
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)
# </YOUR CODE>

In [89]:
train_val_loaders = {
    "train": train_dataloader,
    "valid": valid_dataloader
}

runner = SupervisedRunner(
    input_key=(
        "input_ids",
        "attention_mask",
        "token_type_ids"
    )
)

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=train_val_loaders,
    callbacks=[
        AccuracyCallback(num_classes=num_labels),
        SchedulerCallback(mode='batch'),
    ],
    logdir=log_dir,
    num_epochs=epochs,
    verbose=False,
)

/home/care1e55/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



[2020-05-11 14:31:05,336] 
1/10 * Epoch 1 (_base): lr=9.474e-06 | momentum=0.9000
1/10 * Epoch 1 (train): accuracy01=0.7044 | loss=0.5368 | lr=7.369e-06 | momentum=0.9000
1/10 * Epoch 1 (valid): accuracy01=0.7622 | loss=0.4592


/home/care1e55/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning:

Please also save or load the state of the optimzer when saving or loading the scheduler.



[2020-05-11 14:35:28,864] 
2/10 * Epoch 2 (_base): lr=8.421e-06 | momentum=0.9000
2/10 * Epoch 2 (train): accuracy01=0.7722 | loss=0.4471 | lr=8.947e-06 | momentum=0.9000
2/10 * Epoch 2 (valid): accuracy01=0.7891 | loss=0.4213
[2020-05-11 14:39:54,691] 
3/10 * Epoch 3 (_base): lr=7.368e-06 | momentum=0.9000
3/10 * Epoch 3 (train): accuracy01=0.7907 | loss=0.4204 | lr=7.895e-06 | momentum=0.9000
3/10 * Epoch 3 (valid): accuracy01=0.7970 | loss=0.4071
[2020-05-11 14:44:20,374] 
4/10 * Epoch 4 (_base): lr=6.316e-06 | momentum=0.9000
4/10 * Epoch 4 (train): accuracy01=0.8030 | loss=0.4028 | lr=6.842e-06 | momentum=0.9000
4/10 * Epoch 4 (valid): accuracy01=0.7943 | loss=0.4114
[2020-05-11 14:48:45,891] 
5/10 * Epoch 5 (_base): lr=5.263e-06 | momentum=0.9000
5/10 * Epoch 5 (train): accuracy01=0.8113 | loss=0.3916 | lr=5.789e-06 | momentum=0.9000
5/10 * Epoch 5 (valid): accuracy01=0.8033 | loss=0.3987
[2020-05-11 14:53:09,986] 
6/10 * Epoch 6 (_base): lr=4.211e-06 | momentum=0.9000
6/10 * Epo

In [90]:
logits = runner.predict_loader(model=model, loader=test_dataloader, resume=log_dir+'checkpoints/best.pth')
y_pred = []
counter = 0
softmax = torch.nn.Softmax(dim=1)
for i in logits:
    for j in softmax(i['logits']).argmax(axis=1).tolist():
        y_pred.append(j)

y_true = []

for i in test_dataloader:
    for j in i['targets'].tolist():
        y_true.append(j)
        

accuracy_score(y_true, y_pred)

0.815659755765277

# Используем google/bert_uncased_L-4_H-256_A-4 на QQP

In [27]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 2

# <YOUR CODE>
pretrained_model_name = 'google/bert_uncased_L-4_H-256_A-4'
tokenizer = transformers.BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification(pretrained_model_name, num_labels=num_labels)
# </YOUR CODE>

model.to(device)
print("Success!")
seed = 404
set_global_seed(seed)
prepare_cudnn(True)
# Гиперпараметры для обучения модели. Подбери нужные для каждой модели.

epochs = 10
lr = 1e-5
warmup_steps = len(train_dataloader) // 2
t_total = len(train_dataloader) * epochs

optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters()], "weight_decay": 0.0},
]

# <YOUR CODE>
criterion = torch.nn.CrossEntropyLoss()
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)
# </YOUR CODE>

Success!


In [28]:
train_val_loaders = {
    "train": train_dataloader,
    "valid": valid_dataloader
}

runner = SupervisedRunner(
    input_key=(
        "input_ids",
        "attention_mask",
        "token_type_ids"
    )
)

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=train_val_loaders,
    callbacks=[
        AccuracyCallback(num_classes=num_labels),
        SchedulerCallback(mode='batch'),
    ],
    logdir=log_dir,
    num_epochs=epochs,
    verbose=False,
)

/home/care1e55/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



[2020-05-11 01:33:30,457] 
1/10 * Epoch 1 (_base): lr=9.474e-06 | momentum=0.9000
1/10 * Epoch 1 (train): accuracy01=0.7539 | loss=0.4657 | lr=7.369e-06 | momentum=0.9000
1/10 * Epoch 1 (valid): accuracy01=0.8117 | loss=0.3846


/home/care1e55/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning:

Please also save or load the state of the optimzer when saving or loading the scheduler.



[2020-05-11 01:49:53,157] 
2/10 * Epoch 2 (_base): lr=8.421e-06 | momentum=0.9000
2/10 * Epoch 2 (train): accuracy01=0.8272 | loss=0.3669 | lr=8.947e-06 | momentum=0.9000
2/10 * Epoch 2 (valid): accuracy01=0.8467 | loss=0.3322
[2020-05-11 02:06:15,264] 
3/10 * Epoch 3 (_base): lr=7.368e-06 | momentum=0.9000
3/10 * Epoch 3 (train): accuracy01=0.8466 | loss=0.3324 | lr=7.895e-06 | momentum=0.9000
3/10 * Epoch 3 (valid): accuracy01=0.8537 | loss=0.3214
[2020-05-11 02:22:37,421] 
4/10 * Epoch 4 (_base): lr=6.316e-06 | momentum=0.9000
4/10 * Epoch 4 (train): accuracy01=0.8591 | loss=0.3096 | lr=6.842e-06 | momentum=0.9000
4/10 * Epoch 4 (valid): accuracy01=0.8595 | loss=0.3079
[2020-05-11 02:39:04,583] 
5/10 * Epoch 5 (_base): lr=5.263e-06 | momentum=0.9000
5/10 * Epoch 5 (train): accuracy01=0.8686 | loss=0.2925 | lr=5.789e-06 | momentum=0.9000
5/10 * Epoch 5 (valid): accuracy01=0.8651 | loss=0.2985
[2020-05-11 02:55:26,707] 
6/10 * Epoch 6 (_base): lr=4.211e-06 | momentum=0.9000
6/10 * Epo

In [80]:
# test_dataset = TextClassificationDataset(test_pd, tokenizer)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

logits = runner.predict_loader(model=model, loader=test_dataloader, resume=log_dir+'checkpoints/best.pth')
y_pred = []
counter = 0
softmax = torch.nn.Softmax(dim=1)
for i in logits:
    for j in softmax(i['logits']).argmax(axis=1).tolist():
        y_pred.append(j)

y_true = []

for i in test_dataloader:
    for j in i['targets'].tolist():
        y_true.append(j)
        

accuracy_score(y_true, y_pred)

0.8696589135765772

Ииии отчет!

Напиши внизу небольшой отчет о проделанной работе. Ожидается сравнение результатов модели с разным количеством голов/слоев на разных датасетах на `test`. Если для оценки качества на датасете используется необычная метрика(не Accuracy или F1), то можно использовать один из них. Было бы круто, если бы вычислялась нужная метрика и она использовалась в отчете.

<ТВОЙ ОТЧЕТ>

В ходе выполнения дз использовались статьи и документация hugging face, например https://huggingface.co/transformers/main_classes/tokenizer.html. Решались задачи бинарной классификации SST-2 и QQP. Полагая, что классы достаточно сбалансированы, использовалась метрика accuracy. Для QQP в tokenizer подавалась пара пердложений (второе как text_pair), а для SST просто предложение. attention_mask, token_type_ids, input_ids получены из tokenizer (спасибо hugging face за это). Хотелось сначала использовать distiledBert, но в нем нет token_type. Для каждого типа задачи были взяты предобученные Bert'ы модели и токенайзера с 4 и 2 head и 256 и 128 hidden_dim соответвенно, к ним добавлялся линейный классификатор на 2 класса. В результате как и ожидалось в первом случае качество лучше, но дольше обучение и более требовательно к ресурсам. Гиперпараметры - без изменений. Немного пришлось повозиться с DataLoaders, например привести названия переменных в консистентный вид и убрать randomsampler для test.


</ТВОЙ ОТЧЕТ>

In [108]:
report = pd.DataFrame(columns = [
        'model', 
        'task', 
        'train acc',
        'train loss',
        'val acc',
        'val loss',
        'test acc'
    ])
report.loc[0] = ['google/bert_uncased_L-4_H-256_A-4', 'SST-2', '0.9291', '0.1825', '0.9145', '0.2261', '0.8555']
report.loc[1] = ['google/bert_uncased_L-2_H-128_A-2', 'SST-2', '0.8775', '0.2946', '0.8751', '0.3035', '0.8050']
report.loc[2] = ['google/bert_uncased_L-2_H-128_A-2', 'QQP', '0.8268', '0.3661', '0.8163', '0.3806', '0.8156']
report.loc[3] = ['google/bert_uncased_L-4_H-256_A-4', 'QQP', '0.8902', '0.2516', '0.8727', '0.2920', '0.8696']
report

,model,task,train acc,train loss,val acc,val loss,test acc
0,google/bert_uncased_L-4_H-256_A-4,SST-2,0.9291,0.1825,0.9145,0.2261,0.8555
1,google/bert_uncased_L-2_H-128_A-2,SST-2,0.8775,0.2946,0.8751,0.3035,0.8050
2,google/bert_uncased_L-2_H-128_A-2,QQP,0.8268,0.3661,0.8163,0.3806,0.8156
3,google/bert_uncased_L-4_H-256_A-4,QQP,0.8902,0.2516,0.8727,0.2920,0.8696
